# 12.1 编译器和解释器
- **目录**
  - 12.1.1 符号式编程
  - 12.1.2 混合式编程
  - 12.1.3 Sequential的混合式编程
    - 12.1.3.1 通过混合式编程加速
    - 12.1.3.2 序列化

- 目前为止，本书主要关注的是**命令式编程（imperative programming）**。
- 命令式编程使用诸如`print`、“`+`”和`if`之类的语句来更改程序的状态。
- 考虑下面这段简单的命令式程序：

In [17]:
def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g

print(fancy_func(1, 2, 3, 4))

10


- Python是一种**解释型语言（interpreted language）**。
  - 因此，当对上面的`fancy_func`函数求值时，它按顺序执行函数体的操作。
  - 也就是说，它将通过对`e = add(a, b)`求值，并将结果存储为变量`e`，从而更改程序的状态。
  - 接下来的两个语句`f = add(c, d)`和`g = add(e, f)`也将执行类似地操作，即执行加法计算并将结果存储为变量。 
- 图12.1.1说明了数据流。


<center><img src='../img/computegraph.svg'/></center>
<center>图12.1.1 命令式编程中的数据流</center>

- 尽管命令式编程很方便，但可能**效率不高**。
  - 一方面原因，Python会单独执行这三个函数的调用，而没有考虑`add`函数在`fancy_func`中被重复调用。
    - 如果在一个GPU（甚至多个GPU）上执行这些命令，那么Python解释器产生的开销可能会非常大。
  - 此外，它需要保存`e`和`f`的变量值，直到`fancy_func`中的所有语句都执行完毕。
    - 这是因为程序不知道在执行语句`e = add(a, b)`和`f = add(c, d)`之后，其他部分是否会使用变量`e`和`f`。

----------
- **说明：何为命令式编程和解释型语言？**
  - **命令式编程**
    - **定义**：命令式编程是一种编程范式，其中程序是由计算机可以执行的一系列指令组成的。这些指令明确告诉计算机要执行哪些操作。
    - **原理**
      - **顺序执行**: 指令按照特定的顺序逐个执行。
      - **状态变化**: 命令式编程依赖于系统状态的改变。例如上述示例中的变量`总和`就是一个状态。
      - **控制结构**: 使用循环、条件判断等来控制程序的流程。
  - **解释型语言**
      - **定义**：解释型语言是通过解释器**逐行解释和执行**的编程语言，而不是编译成本地机器代码然后执行。每次程序运行时，解释器都会重新解释源代码。
      - **原理**：
        - **逐行解释**: 解释器逐行读取源代码，并将其转换为中间代码或直接执行。
        - **没有独立的编译过程**: 解释型语言通常不需要单独的编译步骤。源代码每次运行时都会被解释。
        - **执行速度**: 通常比编译型语言慢，因为解释过程在运行时发生。
  - **联系**
    - **都可以用于命令式编程**: 解释型语言通常也支持命令式编程范式。例如，上面的Python代码示例就是命令式编程。
    - **开发速度与执行速度的权衡**: 命令式编程的解释型语言通常开发更快，因为你可以立即看到代码的效果，但执行速度可能较慢。
    - **可读性和维护性**: 命令式编程的解释型语言通常具有良好的可读性和维护性，使开发者能够更容易理解和修改代码。
--------------

## 12.1.1 符号式编程

- 考虑另一种选择**符号式编程（symbolic programming）**,即代码通常只在完全定义了过程之后才执行计算。这个策略被多个深度学习框架使用，包括Theano和TensorFlow（后者已经获得了命令式编程的扩展）。
- 一般包括以下步骤：
  - 定义计算流程。
  - 将流程编译成可执行的程序。
  - 给定输入，调用编译好的程序执行。
- 这将允许进行大量的优化。
  -  首先，在大多数情况下，我们可以跳过Python解释器，从而消除因为多个更快的GPU与单个CPU上的单个Python线程搭配使用时产生的性能瓶颈。   
  -  其次，编译器可以将上述代码优化和重写为`print((1 + 2) + (3 + 4))`甚至`print(10)`。
    - 因为编译器在将其转换为机器指令之前可以看到完整的代码，所以这种优化是可以实现的。
    - 例如，只要某个变量不再需要，编译器就可以释放内存（或者从不分配内存），或者将代码转换为一个完全等价的片段。
- 下面，将通过模拟命令式编程来进一步了解符号式编程的概念。

In [18]:
def add_():
    return '''
def add(a, b):
    return a + b
'''

def fancy_func_():
    return '''
def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
'''

def evoke_():
    return add_() + fancy_func_() + 'print(fancy_func(1, 2, 3, 4))'

prog = evoke_()
print(prog)
y = compile(prog, '', 'exec')
exec(y)


def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
print(fancy_func(1, 2, 3, 4))
10


- **命令式（解释型）编程**和**符号式编程**的区别如下：
  * 命令式编程更容易使用。
    - 在Python中，命令式编程的大部分代码都是简单易懂的。
    - 命令式编程也**更容易调试**，这是因为无论是获取和打印所有的中间变量值，或者使用Python的内置调试工具都更加简单。
  * 符号式编程运行效率更高，**更易于移植**。
    - 符号式编程更容易在编译期间优化代码，同时还能够将程序移植到与Python无关的格式中。
    - 从而允许程序在非Python环境中运行，避免了任何潜在的与Python解释器相关的性能问题。

## 12.1.2 混合式编程
- 历史上，大部分深度学习框架都在命令式编程与符号式编程之间进行选择。
- 例如，Theano、TensorFlow（灵感来自前者）、Keras和CNTK采用了符号式编程。
- 相反地，Chainer和**PyTorch采取了命令式编程**。
- 在后来的版本更新中，TensorFlow2.0和Keras增加了命令式编程。

- 如上所述，PyTorch是基于命令式编程并且使用动态计算图。
- 为了能够利用符号式编程的可移植性和效率，开发人员思考能否将这两种编程模型的优点结合起来，于是就产生了torchscript。
  - torchscript允许用户使用纯命令式编程进行开发和调试，同时能够将大多数程序转换为符号式程序，以便在需要**产品级计算性能**和部署时使用。


## 12.1.3 `Sequential`的混合式编程

- 要了解混合式编程的工作原理，最简单的方法是考虑具有多层的深层网络。
- 按照惯例，Python解释器需要执行**所有层**的代码来生成一条指令，然后将该指令转发到CPU或GPU。
- 对于单个的（快速的）计算设备，这不会导致任何重大问题。
- 另一方面，如果我们使用先进的8-GPU服务器，比如AWS P3dn.24xlarge实例，Python将很难让所有的GPU都保持忙碌。
- 在这里，瓶颈是**单线程的Python解释器**。
- 让我们看看如何通过将`Sequential`替换为`HybridSequential`来解决代码中这个瓶颈。
- 首先，我们定义一个简单的多层感知机。


In [19]:
import torch
from torch import nn
from d2l import torch as d2l


# 生产网络的工厂模式
def get_net():
    net = nn.Sequential(nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 2))
    return net

x = torch.randn(size=(1, 512))
net = get_net()
net(x)

tensor([[-0.0655,  0.0201]], grad_fn=<AddmmBackward0>)

- 通过**使用`torch.jit.script`函数**来转换模型，就有能力编译和优化多层感知机中的计算，而模型的计算结果保持不变。


In [20]:
net = torch.jit.script(net)
net(x)

tensor([[-0.0655,  0.0201]], grad_fn=<DifferentiableGraphBackward>)

- 编写与之前相同的代码，再**使用`torch.jit.script`简单地转换模型**，当完成这些任务后，网络就将得到优化（我们将在下面对性能进行基准测试）。


### 12.1.3.1 通过混合式编程加速

- 为了证明通过编译获得了性能改进，比较了混合编程前后执行`net(x)`所需的时间。
- 此处先定义一个度量时间的类，它在本章中在衡量（和改进）模型性能时将非常有用。


In [21]:
#@save
class Benchmark:
    """用于测量运行时间"""
    def __init__(self, description='Done'):
        self.description = description

    def __enter__(self):
        self.timer = d2l.Timer()
        return self

    def __exit__(self, *args):
        print(f'{self.description}: {self.timer.stop():.4f} sec')

- 现在我们可以调用网络两次：
  - 一次使用torchscript
  - 一次不使用torchscript。


In [22]:
net = get_net()
with Benchmark('无torchscript'):
    for i in range(1000): net(x)

net = torch.jit.script(net)
with Benchmark('有torchscript'):
    for i in range(1000): net(x)

无torchscript: 0.4311 sec
有torchscript: 0.1700 sec


- 如以上结果所示，在`nn.Sequential`的实例被函数`torch.jit.script`脚本化后，通过使用**符号式编程**提高了计算性能。
- （个人感觉在这种小模型中，计算性能提高并不明显！）

--------
- **说明：torch.jit.script功能**
  - TorchScript简介：
    - TorchScript是PyTorch的编程语言，它可以**捕获深度学习模型的结构**。它是Python的一种静态可分析和可优化的子集，用于描述包含张量计算的计算图。
  - TorchScript的主要目的：
    - **性能优化**: TorchScript允许PyTorch对代码进行额外优化，提高运行速度。
    - **跨平台部署**: 通过将模型转换为TorchScript，可以脱离Python解释器，在不同的平台上运行模型。
    - **易于导出**: TorchScript格式让你能够将模型导出到其他编程语言和环境。
  - TorchScript的创建方式：
    - 使用脚本：可以通过`torch.jit.script`装饰器或函数将普通的PyTorch代码转换为TorchScript。这样做会进行静态分析并创建一个不依赖Python的可序列化表示。
    - 使用跟踪：使用`torch.jit.trace`可以通过跟踪代码的执行来创建TorchScript。这对于不包含控制流的简单模块非常有用。
  - TorchScript的优缺点：
    - 优点：
      - **优化**: 可以对整个模型进行全局优化。
      - **跨平台**: 可以在没有Python的环境中运行。
      - **混合前端**: 允许在一个模型中混合使用Eager模式（标准PyTorch）和Graph模式（TorchScript）。
    - 缺点：
      - **限制性**: TorchScript只支持Python的一个子集，某些功能可能无法使用。
      - **可能的性能问题**: 正如你提到的那样，某些情况下可能不会看到预期的性能提升。
  - 序列化和反序列化：
    可以通过`torch.jit.save`和`torch.jit.load`轻松保存和加载TorchScript模型。
-----------

### 12.1.3.2 序列化


- 编译模型的好处之一是我们可以将模型及其参数**序列化（保存）** 到磁盘。
- 这允许这些训练好的模型部署到其他设备上，并且还能方便地使用其他前端编程语言。
- 同时，通常编译模型的代码执行速度也比命令式编程更快。
- 让我们看看`save`的实际功能。


In [23]:
net.save('my_mlp')
!ls -lh my_mlp*

-rw-r--r-- 1 hp None 652K Feb 20 12:55 my_mlp


## 小结

* 命令式编程使得新模型的设计变得容易，因为可以依据控制流编写代码，并拥有相对成熟的Python软件生态。
* 符号式编程要求我们先定义并且编译程序，然后再执行程序，其好处是提高了计算性能。
